<h1><center>Project 2 LOS</h1>
<h2><center>Pre-Processing</h2>
<h4><center>TCSS 555<br>
Spring 2018<br>
by: Thuan Lam, Tood Robbins, Inno Estrera</h4></center>


<h2>Libraries</h2>

In [21]:
# Load libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from dateutil.parser import parse
from datetime import datetime

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Load Model Datasets
<h1>Data set descriptions</h1>
<h3>https://mimic.physionet.org/mimictables/admissions/</h3>

In [5]:
import os 
cwd = os.getcwd()
print('Current folder is {}'.format(cwd))

Current folder is C:\Users\toddr\OneDrive\Documents\anaconda-workspace\Project-2-Length-Of-Stay\phase 1


In [6]:
# load data sets

#Note: the data location will need to be updated based on your mimic3 file location
if os.name == "posix":
    CSV = []
    unreadable = set([
        "LABEVENTS.csv",
        "INPUTEVENTS_CV.csv",
        "DATETIMEEVENTS.csv",
        "PRESCRIPTIONS.csv",
        "NOTEVENTS.csv",
        "OUTPUTEVENTS.csv",
        "CHARTEVENTS.csv",
        "D_LABITEMS.csv",
        "CPTEVENTS.csv"
    ])
    for dirpath, dirnames, filenames in os.walk('/Users/innoestrera/Desktop/mimic3'):
        dataSets = filenames
    for fileName in dataSets:
        if fileName != ".DS_Store" and fileName not in unreadable:
            csv = "~/Desktop/mimic3/{}".format(fileName);
            print("Reading {}".format(csv))
            CSV.append(pd.read_csv(csv))
    print("finished loading all data")
else:
    # Top Data
    dataLos = pd.read_csv("D:\mimic3\ICUSTAYS.csv") # contains field we are trying to predict
    dataAdm = pd.read_csv("D:\mimic3\ADMISSIONS.csv") # Number of admissions
    dataPat = pd.read_csv("D:\mimic3\PATIENTS.csv") # Number of patients admitted 
    #
#    dataCallOUT = pd.read_csv("C:\mimic3\CALLOUT.csv")
#    dataCareGiver = pd.read_csv("C:\mimic3\CAREGIVERS.csv")
#    dataChartEvents = pd.read_csv("C:\mimic3\CHARTEVENTS.csv")
#    dataCPTEvents = pd.read_csv("C:\mimic3\CPTEVENTS.csv")
#    dataCPT = pd.read_csv("C:\mimic3\D_CPT.csv")
#    dataICD_Diagnoses = pd.read_csv("C:\mimic3\D_ICD_DIAGNOSES.csv")
#    dataICD_Procedures = pd.read_csv("C:\mimic3\D_ICD_PROCEDURES.csv")
#    dataItems = pd.read_csv("C:\mimic3\D_ITEMS.csv")
#    dataLabelItems = pd.read_csv("C:\mimic3\D_LABITEMS.csv")
#    dataDateTimeEvents = pd.read_csv("C:\mimic3\DATETIMEEVENTS.csv")
#    dataDiagnosis_ICD = pd.read_csv("C:\mimic3\DIAGNOSES_ICD.csv")
#    dataDrugCodes = pd.read_csv("C:\mimic3\DRGCODES.csv")
#    dataCareVueInput = pd.read_csv("C:\mimic3\INPUTEVENTS_CV.csv")
#    dataMetavisionInput = pd.read_csv("C:\mimic3\INPUTEVENTS_MV.csv")
#    dataLabEvents = pd.read_csv("C:\mimic3\LABEVENTS.csv")
#    dataMBEvents = pd.read_csv("C:\mimic3\MICROBIOLOGYEVENTS.csv")
#    dataNotes = pd.read_csv("C:\mimic3\\NOTEEVENTS.csv") # duplicate backlash to avoid unicode command.
#    dataOutputEvents = pd.read_csv("C:\mimic3\OUTPUTEVENTS.csv")
#    dataPrescriptions = pd.read_csv("C:\mimic3\PRESCRIPTIONS.csv")
#    dataProceduresDone = pd.read_csv("C:\mimic3\PROCEDUREEVENTS_MV.csv")
#    dataProceduresCost = pd.read_csv("C:\mimic3\PROCEDURES_ICD.csv")
#    dataServices = pd.read_csv("C:\mimic3\SERVICES.csv")
#    dataTransfers = pd.read_csv("C:\mimic3\TRANSFERS.csv")
    print("done")

done


In [8]:
# shuffle data sets
dataLos = shuffle(dataLos)
dataAdm = shuffle(dataAdm)
dataPat = shuffle(dataPat)

#dataCallOUT = shuffle(dataCallOUT)
#dataCareGiver = shuffle(dataCareGiver)
#dataChartEvents = shuffle(dataChartEvents)
#dataCPTEvents = shuffle(dataCPTEvents)
#dataCPT = shuffle(dataCPT)
#dataICD_Diagnoses = shuffle(dataICD_Diagnoses)
#dataICD_Procedures = shuffle(dataICD_Procedures)
#dataItems = shuffle(dataItems)
#dataLabelItems = shuffle(dataLabelItems)
#dataDateTimeEvents = shuffle(dataDateTimeEvents)
#dataDiagnosis_ICD = shuffle(dataDiagnosis_ICD)
#dataDrugCodes = shuffle(dataDrugCodes)
#dataCareVueInput = shuffle(dataCareVueInput)
#dataMetavisionInput = shuffle(dataMetavisionInput)
#dataLabEvents = shuffle(dataLabEvents)
#dataMBEvents = shuffle(dataMBEvents)
#dataNotes = shuffle(dataNotes)
#dataOutputEvents = shuffle(dataOutputEvents)
#dataPrescriptions = shuffle(dataPrescriptions)
#dataProceduresDone = shuffle(dataProceduresDone)
#dataProceduresCost = shuffle(dataProceduresCost)
#dataServices = shuffle(dataServices)
#dataTransfers = shuffle(dataTransfers)
print("shuffle")

shuffle


In [9]:
# lower case all columns 
dataLos.columns = [x.lower() for x in dataLos.columns]
dataAdm.columns = [x.lower() for x in dataAdm.columns]
dataPat.columns = [x.lower() for x in dataPat.columns]

#dataCallOUT.columns = [x.lower() for x in dataCallOUT.columns]
#dataCareGiver.columns = [x.lower() for x in dataCareGiver.columns]
#dataChartEvents.columns = [x.lower() for x in dataChartEvents.columns]
#dataCPTEvents.columns = [x.lower() for x in dataCPTEvents.columns]
#dataCPT.columns = [x.lower() for x in dataCPT.columns]
#dataICD_Diagnoses.columns = [x.lower() for x in dataICD_Diagnoses.columns]
#dataICD_Procedures.columns = [x.lower() for x in dataICD_Procedures.columns]
#dataItems.columns = [x.lower() for x in dataItems.columns]
#dataLabelItems.columns = [x.lower() for x in dataLabelItems.columns]
#dataDateTimeEvents.columns = [x.lower() for x in dataDateTimeEvents.columns]
#dataDiagnosis_ICD.columns = [x.lower() for x in dataDiagnosis_ICD.columns]
#dataDrugCodes.columns = [x.lower() for x in dataDrugCodes.columns]
#dataCareVueInput.columns = [x.lower() for x in dataCareVueInput.columns]
#dataMetavisionInput.columns = [x.lower() for x in dataMetavisionInput.columns]
#dataLabEvents.columns = [x.lower() for x in dataLabEvents.columns]
#dataMBEvents.columns = [x.lower() for x in dataMBEvents.columns]
#dataNotes.columns = [x.lower() for x in dataNotes.columns]
#dataOutputEvents.columns = [x.lower() for x in dataOutputEvents.columns]
#dataPrescriptions.columns = [x.lower() for x in dataPrescriptions.columns]
#dataProceduresDone.columns = [x.lower() for x in dataProceduresDone.columns]
#dataProceduresCost.columns = [x.lower() for x in dataProceduresCost.columns]
#dataServices.columns = [x.lower() for x in dataServices.columns]
#dataTransfers.columns = [x.lower() for x in dataTransfers.columns]

In [10]:
# print table information
print("========ICU STAYS========")
dataLos.info()
print("=========================")
print()
print("========ADMISSIONS=======")
dataAdm.info()
print("=========================")
print()
print("========PATIENTS=========")
dataPat.info()
print("=========================")
print()
#print("========CallOUT=========")
#dataCallOUT.info()
#print("=========================")
#print()
#print("========CareGiver=========")
#dataCareGiver.info()
#print("=========================")
#print()
#print("========ChartEvents=========")
#dataChartEvents.info()
#print("=========================")
#print()
#print("========CPTEvents=========")
#dataCPTEvents.info()
#print("=========================")
#print()
#print("========CPT=========")
#dataCPT.info()
#print("=========================")
#print()
#print("========ICD_Diagnoses=========")
#dataICD_Diagnoses.info()
#print("=========================")
#print()
#print("========ICD_Procedures=========")
#dataICD_Procedures.info()
#print("=========================")
#print()
#print("========Items=========")
#dataItems.info()
#print("=========================")
#print()
#print("========LabelItems=========")
#dataLabelItems.info()
#print("=========================")
#print()
#print("========DateTimeEvents=========")
#dataDateTimeEvents.info()
#print("=========================")
#print()
#print("========Diagnosis_ICD=========")
#dataDiagnosis_ICD.info()
#print("=========================")
#print()
#print("========DrugCodes=========")
#dataDrugCodes.info()
#print("=========================")
#print()
#print("========CareVueInput=========")
#dataCareVueInput.info()
#print("=========================")
#print()
#print("========MetavisionInput=========")
#dataMetavisionInput.info()
#print("=========================")
#print()
#print("========LabEvents=========")
#dataLabEvents.info()
#print("=========================")
#print()
#print("========MBEvents=========")
#dataMBEvents.info()
#print("=========================")
#print()
#print("========Notes=========")
#dataNotes.info()
#print("=========================")
#print()
#print("========OutputEvents=========")
#dataOutputEvents.info()
#print("=========================")
#print()
#print("========Prescriptions=========")
#dataPrescriptions.info()
#print("=========================")
#print()
#print("========ProceduresDone=========")
#dataProceduresDone.info()
#print("=========================")
#print()
#print("========ProceduresCost=========")
#dataProceduresCost.info()
#print("=========================")
#print()
#print("========Services=========")
#dataServices.info()
#print("=========================")
#print()
#print("========Transfers=========")
#dataTransfers.info()
#print("=========================")

========ICU STAYS========
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61532 entries, 16183 to 40796
Data columns (total 12 columns):
row_id            61532 non-null int64
subject_id        61532 non-null int64
hadm_id           61532 non-null int64
icustay_id        61532 non-null int64
dbsource          61532 non-null object
first_careunit    61532 non-null object
last_careunit     61532 non-null object
first_wardid      61532 non-null int64
last_wardid       61532 non-null int64
intime            61532 non-null object
outtime           61522 non-null object
los               61522 non-null float64
dtypes: float64(1), int64(6), object(5)
memory usage: 6.1+ MB

========ADMISSIONS=======
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 27646 to 54237
Data columns (total 19 columns):
row_id                  58976 non-null int64
subject_id              58976 non-null int64
hadm_id                 58976 non-null int64
admittime               58976 non-null object
dis

## Examine datasets

In [11]:
dataLos.sample(n=20)

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
3147,3551,2818,173102,289838,carevue,NICU,NICU,56,56,2160-03-11 20:02:33,2160-03-11 22:51:00,0.1170
35548,33636,26505,172940,266473,carevue,NICU,NICU,56,56,2194-12-20 18:16:22,2194-12-20 20:44:35,0.1029
58684,59074,92775,197806,247620,metavision,MICU,MICU,52,52,2146-10-07 00:15:55,2146-10-08 18:24:23,1.7559
34272,31873,25155,123755,209654,metavision,CSRU,CSRU,15,15,2116-11-09 11:44:12,2116-11-11 18:12:22,2.2696
58918,60433,96704,195113,277125,metavision,CCU,CCU,7,7,2158-09-13 18:58:08,2158-09-14 08:21:09,0.5577
39629,42816,44523,142963,253958,metavision,MICU,MICU,52,52,2127-05-29 16:33:04,2127-05-30 19:58:51,1.1429
56651,56644,85350,142413,232605,metavision,MICU,MICU,52,52,2124-01-26 20:43:05,2124-01-27 15:06:49,0.7665
20723,20050,15843,192396,261649,carevue,CCU,CCU,7,7,2102-08-31 13:35:54,2102-09-01 14:43:52,1.0472
15864,13692,10769,120240,224318,carevue,MICU,MICU,50,50,2133-03-27 09:14:10,2133-03-28 19:52:12,1.4431
56094,54298,78204,123171,244704,metavision,SICU,SICU,57,57,2197-11-01 13:48:20,2197-11-09 13:06:14,7.9708


In [12]:
dataAdm.sample(n=20)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
9726,10670,8729,101442,2150-11-23 20:26:00,2150-11-29 13:36:00,NaN,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Government,NaN,PROTESTANT QUAKER,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NEWBORN,0,1
40080,41312,44894,148499,2132-07-06 04:31:00,2132-07-16 17:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2132-07-06 00:04:00,2132-07-06 08:00:00,FEVER;NEUTROPENIA;SAH,0,1
20646,21366,17489,165441,2142-07-08 15:33:00,2142-07-11 13:39:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2142-07-08 11:38:00,2142-07-08 15:39:00,S/P FALL,0,1
28126,28202,23110,165634,2196-02-17 15:15:00,2196-02-25 12:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,PROTESTANT QUAKER,WIDOWED,OTHER,NaN,NaN,MITRAL REGURGITATION,0,1
39830,36888,30287,147245,2189-09-23 17:56:00,2189-09-28 21:25:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SHORT TERM HOSPITAL,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2189-09-23 12:58:00,2189-09-23 18:51:00,PNEUMONIA,0,1
17214,17637,14403,159386,2200-05-02 18:13:00,2200-05-06 12:22:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,UNOBTAINABLE,NaN,WHITE,NaN,NaN,NEWBORN,0,1
2320,2144,1772,198360,2151-11-29 20:24:00,2151-12-01 11:15:00,NaN,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,UNOBTAINABLE,SINGLE,ASIAN,NaN,NaN,HYPERBILIRUBINEMIA,0,1
3302,3161,2605,148903,2144-08-22 22:54:00,2144-08-24 16:24:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISCH-TRAN TO PSYCH HOSP,Medicaid,ENGL,UNOBTAINABLE,SINGLE,WHITE,2144-08-22 21:15:00,2144-08-23 01:10:00,OVERDOSE,0,1
46840,43824,52622,157538,2158-07-03 19:01:00,2158-07-20 14:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,2158-07-03 13:45:00,2158-07-03 22:41:00,HYPONATREMIA,0,1
44986,45407,57558,177766,2110-05-08 01:17:00,2110-05-13 13:17:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,WHITE,2110-05-07 21:33:00,2110-05-08 02:27:00,LUDWIG'S ANGINA,0,1


In [13]:
dataPat.sample(n=20)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
30178,32792,47660,F,2065-02-17 00:00:00,2144-02-13 00:00:00,NaN,2144-02-13 00:00:00,1
30743,30248,32212,F,1858-10-27 00:00:00,2158-11-10 00:00:00,2158-11-10 00:00:00,NaN,1
15747,19253,20406,M,2145-12-04 00:00:00,NaN,NaN,NaN,0
23415,23319,24715,M,2192-05-01 00:00:00,NaN,NaN,NaN,0
16253,18894,20017,M,2179-11-19 00:00:00,NaN,NaN,NaN,0
29265,34853,55301,F,2149-10-18 00:00:00,NaN,NaN,NaN,0
39204,41082,79154,M,2133-10-19 00:00:00,NaN,NaN,NaN,0
41804,44149,90901,M,2055-12-01 00:00:00,NaN,NaN,NaN,0
1463,3462,3662,M,2125-11-05 00:00:00,NaN,NaN,NaN,0
4354,5062,5342,F,2079-12-16 00:00:00,2122-02-19 00:00:00,2122-02-19 00:00:00,NaN,1


In [17]:
## Analyze Call out data

In [20]:
#if os.name == "posix":
dataCallOUT = pd.read_csv("D:\mimic3\CALLOUT.csv")
dataCallOUT = shuffle(dataCallOUT)
dataCallOUT.columns = [x.lower() for x in dataCallOUT.columns]
print("========CallOUT=========")
dataCallOUT.info()
print("=========================")
print()
print("========Sample=========")
dataCallOUT.sample(n=20)

========CallOUT=========
<class 'pandas.core.frame.DataFrame'>
Int64Index: 34499 entries, 29167 to 17682
Data columns (total 24 columns):
row_id                    34499 non-null int64
subject_id                34499 non-null int64
hadm_id                   34499 non-null int64
submit_wardid             34498 non-null float64
submit_careunit           5122 non-null object
curr_wardid               34498 non-null float64
curr_careunit             34498 non-null object
callout_wardid            34499 non-null int64
callout_service           34499 non-null object
request_tele              34499 non-null int64
request_resp              34499 non-null int64
request_cdiff             34499 non-null int64
request_mrsa              34499 non-null int64
request_vre               34499 non-null int64
callout_status            34499 non-null object
callout_outcome           34499 non-null object
discharge_wardid          29967 non-null float64
acknowledge_status        34499 non-null object
creat

,row_id,subject_id,hadm_id,submit_wardid,submit_careunit,curr_wardid,curr_careunit,callout_wardid,callout_service,request_tele,...,callout_status,callout_outcome,discharge_wardid,acknowledge_status,createtime,updatetime,acknowledgetime,outcometime,firstreservationtime,currentreservationtime
27578,25957,72173,110037,33.0,NaN,55.0,SICU,1,NSURG,0,...,Inactive,Discharged,55.0,Acknowledged,2169-04-08 08:19:50,2169-04-08 08:24:16,2169-04-08 08:25:37,2169-04-08 17:40:09,2169-04-08 08:55:09,NaN
26773,27844,78215,128835,52.0,MICU,52.0,MICU,1,OMED,1,...,Inactive,Cancelled,NaN,Acknowledged,2120-03-17 14:14:21,2120-03-17 16:19:02,2120-03-17 14:24:54,2120-03-17 16:19:02,NaN,NaN
24087,22682,61587,144575,50.0,NaN,17.0,MICU,1,MED,1,...,Inactive,Discharged,17.0,Acknowledged,2191-11-26 07:56:35,2191-11-27 13:39:03,2191-11-27 13:47:39,2191-11-27 18:10:19,2191-11-27 13:55:18,NaN
17267,15146,32002,106209,14.0,NaN,40.0,TSICU,40,TRAUM,0,...,Inactive,Discharged,40.0,Acknowledged,2126-01-22 09:07:55,2126-01-22 09:07:55,2126-01-22 09:15:04,2126-01-22 18:55:04,NaN,NaN
19462,20698,55163,167775,52.0,NaN,24.0,MICU,24,MED,1,...,Inactive,Discharged,24.0,Acknowledged,2103-01-27 10:46:20,2103-01-27 10:46:20,2103-01-27 11:02:48,2103-01-27 16:25:23,2103-01-27 12:55:23,NaN
17058,15835,32791,180725,14.0,TSICU,14.0,TSICU,1,TRAUM,1,...,Inactive,Cancelled,NaN,Acknowledged,2169-01-01 09:30:33,2169-01-02 21:28:38,2169-01-02 11:50:15,2169-01-02 21:28:38,NaN,NaN
6951,6329,16509,165518,7.0,CCU,7.0,CCU,1,MED,0,...,Inactive,Discharged,0.0,Acknowledged,2143-09-25 11:10:14,2143-09-25 11:10:14,2143-09-25 12:23:37,2143-09-25 13:55:08,NaN,NaN
26377,29695,84223,173847,33.0,NaN,31.0,SICU,31,NSURG,0,...,Inactive,Discharged,31.0,Acknowledged,2135-12-19 11:23:43,2135-12-19 11:23:43,2135-12-19 11:49:28,2135-12-19 15:10:29,2135-12-19 11:27:03,NaN
232,326,711,168530,50.0,MICU,50.0,MICU,1,MED,1,...,Inactive,Cancelled,NaN,Acknowledged,2184-10-17 10:32:34,2184-10-19 09:05:34,2184-10-17 11:16:19,2184-10-19 09:05:34,NaN,NaN
21787,20405,54420,180446,14.0,TSICU,14.0,TSICU,31,NSURG,0,...,Inactive,Cancelled,NaN,Acknowledged,2159-01-29 12:52:58,2159-01-29 15:55:10,2159-01-29 13:28:35,2159-01-29 15:55:10,2159-01-29 13:42:20,2159-01-29 13:42:20


## Test and Training sets

## Conclusion
#### From the the following tables (ADMISSIONS, ICUSTAYS, PATIENTS) this data concludes that the following columns are the most useful in determining the length of stay of the patient. 